# Import and View Data

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
data = pd.read_csv("../data/Case3HistoricalPrices.csv")
data = data.drop(columns=['Unnamed: 0'])
data.head()

,S1,S2,S3,S4,S5,S6,S7,S8,B1,B2,...,B7,B8,C1,C2,C3,C4,C5,C6,C7,C8
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,98.355850,98.307641,95.630621,98.296988,98.474008,95.663069,98.342292,98.416837,100.086605,99.685232,...,99.674428,100.131478,95.854652,95.571202,94.085760,93.956485,94.140624,94.058865,95.786959,94.331430
2,96.766305,97.785655,95.211317,97.876191,96.910785,95.299426,97.862363,96.850881,100.595828,99.614514,...,99.463006,100.594669,87.500105,86.517650,88.861746,89.205952,89.255488,89.203200,86.873750,89.644765
3,99.498201,97.762932,95.831525,97.979482,99.992847,95.890271,97.890721,99.523488,100.687198,99.425275,...,99.220046,100.639175,84.500885,83.306741,86.183137,86.617107,86.643905,86.512592,83.788306,87.068357
4,96.259221,92.942010,94.272932,92.958011,96.469517,94.318577,93.007052,96.278064,101.939638,100.099827,...,99.843072,101.823484,79.974708,78.434282,80.585967,81.404902,81.484117,81.131945,79.157682,81.778226


# The Allocate Function

# Results

Results are calculated with the annualized daily sharpe ratio and a risk-free rate of 0.

$$S_p = \frac{E[R_p]}{\sigma_p}\cdot\sqrt{252}$$

Where $R_p$ is the daily excess return, calculated as the value of the portfolio today (today's asset prices * weights allocated yesterday) minus the value of the portfolio yesterday (yesterday's asset prices * weights allocated the day before yesterday).

$$R_p = (p_{t}*w_{t}) - (p_{t-1}*w_{t-1})$$

The expected value and standard deviations are found over the course of 1 year, or 252 trading days each. 

The total points is simply the sum of all the annual sharpe ratios.

### Method 1:

We calculate the weights based on a moving average window, continuously updated day by day as we obtain more asset prices. 
On each day, we calculate the z-score of each asset price compared to its price in the last 90 days:
$$z = \frac{x-\mu}{\sigma}$$
If the z-score is greater than 1, that means that our stock price is more than 1 standard deviation above the mean, so it is significantly overpriced. We sell. 
If the z-score is less than 1, that means that our stock price is more than 1 standard deviation below the mean, so it is significantly underpriced. We buy. 

Hyperparameters:
- window: the number of days to consider in the moving average. Default: 90.
- bound: the number of standard deviations away from the mean at which we sell/buy. Default: 1.

In [3]:
# Initialize global variables
window = 90
bound = 1
window_prices = [data[col][:window].to_list() for col in data.columns]
moving_average = [sum(prices)/window for prices in window_prices]
moving_std = [np.std(prices) for prices in window_prices]
portfolio = np.zeros(len(moving_average))

def allocate1(asset_prices):
    for i in range(len(asset_prices)):
        # Calculate z-score
        z = (asset_prices[i] - moving_average[i])/moving_std[i]
        # If z-score is greater than 1, sell
        if z > bound:
            portfolio[i] = max(portfolio[i] - 1, 0)
        # If z-score is less than -1, buy
        elif z < bound:
            portfolio[i] += 1
        # Update window_prices and moving_average
        window_prices[i].pop(0)
        window_prices[i].append(asset_prices[i])
        moving_average[i] = sum(window_prices[i])/window
        moving_std[i] = np.std(window_prices[i])
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

In [4]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = [0]*window

for index, row in data.iterrows():
    if index < window: # Skip training data
        continue
    if index == window: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate1(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) 
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate1(asset_prices)

In [5]:
total = 0
for i in range(1,11):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.9238294422086585
Sharpe ratio for year  2 :  0.22451428387894518
Sharpe ratio for year  3 :  2.222664563631535
Sharpe ratio for year  4 :  1.3872550312807899
Sharpe ratio for year  5 :  0.5611328252243699
Sharpe ratio for year  6 :  -1.1162306514599525
Sharpe ratio for year  7 :  1.3129837733597445
Sharpe ratio for year  8 :  0.3871942037635069
Sharpe ratio for year  9 :  1.1070078503758178
Sharpe ratio for year  10 :  0.4421340420759376
Total points:  7.452485364339354


### Method 2:

Instead of having the moving average be of the previous prices of the given stock, we can calculate the moving average of all commodities/stocks/bonds in the dataset and use it as a market return. This approach has the advantage of being market-neutral, so that we do not treat stocks as undervalued just because the market as a whole is falling, or overvalued when the market is rising. 

If a stock's z-score is greater than the market z-score by a bound, then it is overpriced and we sell. If a stock's z-score is greater than the market z-score by a certain bound, then it is underpriced and we buy. 

Hyperparameters:
- window: the number of days to consider in the moving average. Default: 90.
- bound: the number of standard deviations away from the mean at which we sell/buy. Default: 1.

In [6]:
# Initialize global variables
window = 90
bound = 0
window_prices = [data[col][:window].to_list() for col in data.columns]
moving_average = [sum(prices)/window for prices in window_prices]
moving_std = [np.std(prices) for prices in window_prices]
portfolio = np.zeros(len(moving_average))

def allocate2(asset_prices):
    market_prices = [sum(np.array(window_prices).T[i]) for i in range(window)]
    market_average = sum(market_prices)/window
    market_std = np.std(market_prices)
    market_z = (sum(asset_prices) - market_average) / market_std
    for i in range(len(asset_prices)):
        # Calculate z-score
        z = (asset_prices[i] - moving_average[i])/moving_std[i]
        # If z-score is greater than market_z, sell
        if z > market_z + bound:
            portfolio[i] = max(portfolio[i] - 1, 0)
        # If z-score is less than market_z, buy
        elif z < market_z - bound:
            portfolio[i] += 1
        # Update window_prices and moving_average
        window_prices[i].pop(0)
        window_prices[i].append(asset_prices[i])
        moving_average[i] = sum(window_prices[i])/window
        moving_std[i] = np.std(window_prices[i])
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

In [7]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = [0]*window

for index, row in data.iterrows():
    if index < window: # Skip training data
        continue
    if index == window: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate2(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate2(asset_prices)

In [8]:
total = 0
for i in range(1,11):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  -6.5048100146884416
Sharpe ratio for year  2 :  -12.576133303480159
Sharpe ratio for year  3 :  -2.0694324570016605
Sharpe ratio for year  4 :  -0.9592458076246648
Sharpe ratio for year  5 :  -1.6777197499954133
Sharpe ratio for year  6 :  -5.491375454769156
Sharpe ratio for year  7 :  -0.34913638033261
Sharpe ratio for year  8 :  -2.971275933038518
Sharpe ratio for year  9 :  0.19614726233362853
Sharpe ratio for year  10 :  -1.2194735420020504
Total points:  -33.62245538059904


### Method 3:

Hypothesizing that the worst-performing stocks last period will do better this period (that is, they are likely to be undervalued) and vice versa, we go long in stocks that performed poorly and short in stocks that performed well. Then, every week (or some other timeframe), we rebalance our portfolio by moving the top 20% performing to the bottom 20% performing. If a stock is in neither of those quintiles, we do not include it in our portfolio. 

The difference between this and methods 1 and 2 is that instead of comparing stock prices to the mean, we are comparing them to the returns of other stocks. So, all stocks could be performing better than the mean, but we strictly select the best ones.

For example, if stocks A, B, C has had returns [-0.1, 0.1, 0.5] and our portfolio weights are [0.4, 0.5, 0.1] then we would rebalance the weights to be [0.2, 0.4, 0.0] for the next week. 

We start with an equal dsitribution of weights across each stock.

Hyperparameters:
- timeframe: the period with which we rebalance the portfolio. Default: 5.
- percentile: the top/bottom percentile stocks to remove. Default: 0.2.

In [9]:
timeframe = 5
percentile = 0.2

days = 0
prev_prices = []
prev_weights = np.array([1/24] * 24)

def allocate3(asset_prices):
    global days
    global prev_prices
    if days == 0:
        prev_prices = asset_prices
    if days < timeframe:
        days += 1
        return prev_weights
    days = 0
    returns = (asset_prices - prev_prices)/prev_prices
    remaining = percentile
    while remaining:
        # Search for index of largest returns
        largest, large_ret = 0, -100000
        for i in range(len(returns)):
            if returns[i] > large_ret and prev_weights[i] != 0:
                largest = i
        largest_weight = prev_weights[largest]
        remaining = max(remaining-largest_weight, 0)
        prev_weights[largest] = max(largest_weight-remaining, 0)
    prev_weights[np.argmin(returns)] += percentile
    return prev_weights

In [10]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in data.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate3(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate3(asset_prices)

In [11]:
total = 0
for i in range(1,11):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.33440027002105754
Sharpe ratio for year  2 :  0.39993528413432866
Sharpe ratio for year  3 :  2.1021840912338297
Sharpe ratio for year  4 :  1.101464241109958
Sharpe ratio for year  5 :  0.8618443276593081
Sharpe ratio for year  6 :  -0.807256888639011
Sharpe ratio for year  7 :  1.1661136661727856
Sharpe ratio for year  8 :  -0.12593520601947683
Sharpe ratio for year  9 :  1.2169177703245977
Sharpe ratio for year  10 :  -0.05437084948672699
Total points:  6.19529670651065
